In [ ]:
# Setting up CoinMarketCap API requests w/ my unique API Key
# If you're using my code from GitHub, make sure you change your data limit to allow max amounts -JDrummond 
    # from the command line:
    # jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10
# For the sake of this demo, I'm setting my API 'limit' to 5 just to keep my RAM unused

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '09bfc317-0595-49ac-8a8b-3c9766fa29e4',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
import pandas as pd

pd.json_normalize(data['data'])

In [ ]:
pd.set_option('display.max_columns', None)
pd.json_normalize(data['data'])

In [ ]:
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now', utc=True)
df

In [ ]:
def api_runner():
    global df
    from requests import Request, Session
    from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
    import json

    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'5',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '09bfc317-0595-49ac-8a8b-3c9766fa29e4',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now', utc=True)
    df
    
    # write this pulled data into a .csv file 
    
    if not os.path.isfile(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv'):
        df.to_csv(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv', header = 'columns_names')
    else:
       df.to_csv(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv', mode = 'a', header=False) 

In [ ]:
import os
from time import time 
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner Pull Completed')
    sleep(10)
exit()

In [ ]:
df_csv = pd.read_csv(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv')
df_csv

In [ ]:
pd. set_option('display.float_format', lambda x: '%.5f' % x)
df_csv

In [ ]:
df_grouped = df_csv.groupby('name', sort=False)[['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d']].mean()
df_grouped

In [ ]:
df_stack = df_grouped.stack()
df_stack

In [ ]:
df_DataFrames = df_stack.to_frame(name='values')
df_DataFrames

In [ ]:
index = pd.Index(range(30))

df_index = df_DataFrames.reset_index()
df_index

In [ ]:
df_rename = df_index.rename(columns={'level_1': 'percent_change'})
df_rename

In [ ]:
df_rename['percent_change'] = df_rename['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'], ['1h', '24h', '7d', '30d', '60d', '90d'])
df_rename

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.catplot(x='percent_change', y='values', hue='name', data=df_rename, kind='point')
plt.show()

In [ ]:
df_price = df_csv[['name', 'quote.USD.price', 'timestamp']]
df_price

In [ ]:
df_price = df_csv[['name', 'quote.USD.price', 'timestamp']]
df_price = df_price.query("name == 'Bitcoin'")
df_price

In [ ]:
sns.set_theme(style='darkgrid')

sns.lineplot(x='timestamp', y='quote.USD.price', data=df_price)
plt.show()